In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
import shutil
from fastapi import FastAPI
from pydantic import BaseModel
from pyspark.sql import SparkSession
import uvicorn
# import mysql.connector

# Create the FastAPI app
app = FastAPI()

# MySQL Connection Function
# def get_db_connection():
#     return mysql.connector.connect(
#         host="localhost",      # Change if MySQL is on a different server
#         user="root",  # Replace with your MySQL username
#         password="Root@1234",  # Replace with your MySQL password
#         database="project"
#     )

# Define the input data structure
class InputString(BaseModel):
    input_text: str

# Endpoint to receive and modify the string
@app.post("/modify-string/")
async def modify_string(data: InputString):
    input_data = data.input_text
#     res = ollama.generate(
#         model = "duckdb-nsql",
#         system = "",
#         prompt = input_text
#     )
#     sql_query = res.response
    sql_query = "SELECT * FROM temp_view where Incubation_Center = 'AIC@36Inc' "

    # Spark
    spark = SparkSession.builder.appName("FastAPI-Spark").getOrCreate()
    
    # Read CSV file into DataFrame
    path = "file:///home/talentum/Project/Datasets/"
    file = "Listofstartups.csv"
    file_path = path+file
    df = spark.read.csv(file_path, header=True, inferSchema=True)
        
    # Create a temporary SQL view
    df.createOrReplaceTempView("temp_view")

    # Execute the SQL query
    output_df = spark.sql(sql_query)
    
    # Define output path
    output_path = "file:///home/talentum/Downloads/output.csv"
    
    # Save DataFrame to a CSV file (overwrite existing file)
    output_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)

    # Move and rename the output file
    output_dir = "/home/talentum/Downloads/output.csv"
    files = os.listdir(output_dir)
    output_file = "/home/talentum/Downloads/output_file.csv"
    for file in files:
        if file.startswith("part-"):  # Identify the actual CSV file
            shutil.move(os.path.join(output_dir, file), output_file)

    # Remove the directory after renaming the file
    shutil.rmtree(output_dir)
    
    
    # Insert the result into MySQL database
#     try:
#         conn = get_db_connection()
#         cursor = conn.cursor()
#         sql_insert = """
#             INSERT INTO results (input_file, input_data, sqlquery, output_file_path, datetime)
#             VALUES (%s, %s, %s, %s, %s)
#         """
#         values = (
#             "Listofstartups.csv",  # Input file name
#             input_data,
#             sql_query,
#             output_file,
#             datetime.now()
#         )
#         cursor.execute(sql_insert, values)
#         conn.commit()
#         cursor.close()
#         conn.close()
#     except Exception as e:
#         return {"error": f"Database error: {str(e)}"}
    
    
    
    return {
        "message": "Query executed successfully!",
        "input": input_data,
        "sqlquery": sql_query,
        "output_file": output_file,
    }

# Run the app using uvicorn
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [4958]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:44388 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:44388 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60032 - "POST /modify-string/ HTTP/1.1" 200 OK
